In [ ]:
import glob
import os
from pathlib import Path
import json
import subprocess
from tqdm import tqdm
import jsonlines

In [ ]:
video_dir = os.path.expanduser("/Users/dxy/Desktop/warp/imerit_26102023_clips/")

In [ ]:
vids_1 = glob.glob(f"{video_dir}/*/*.MOV")
vids_2 = glob.glob(f"{video_dir}/*/*/*.MOV")

print(len(vids_1))
print(len(vids_2))

all_vids = vids_1 + vids_2
all_vids = [Path(p) for p in all_vids]
all_vids.sort()
print(f"Total num vids: {len(all_vids)}")
# for p in all_vids:
#     print(p)

In [ ]:
every_nth_frame = 10
dry_run = False
skip_ffmpeg = True
expected_frames = 90

dataset_manifest = []

for seq_num, p in tqdm(enumerate(all_vids)):
#     print(f"Basics -----------")
#     print(p)
#     print(p.parent)
#     print(p.stem)
    
    # create a directory to hold frames
    frame_dir = p.parent / p.stem
    s3_dir = str(frame_dir).replace("/Users/dxy/Desktop/warp", "s3://whoi-rsi-fish-detection/datasets") + "/"
    if not dry_run:
        os.makedirs(frame_dir, exist_ok=True)
    
    # form ffmpeg command to turn into frames
    ffmpeg_cmd = f'ffmpeg -i "{str(p)}" -vf "select=not(mod(n\,{every_nth_frame}))" -vsync vfr -start_number 0 "{frame_dir}/frame_%03d.png"'
    if not dry_run and not skip_ffmpeg:
        os.system(ffmpeg_cmd)
    
    # make sure it generated expected_frames frames
    frame_fps = glob.glob(f"{str(frame_dir)}/*.png")
    if len(frame_fps) != expected_frames:
        print(f"[Warning] Did not create {expected_frames} frames! {str(p)}*****")
    
    # create manifest for aws
    frame_list_dict = [{
        "frame-no": frame_num, 
        "frame": f"frame_{str(frame_num).zfill(3)}.png"
    } for frame_num in range(expected_frames)]
    
    seq_path = frame_dir / f"{p.stem}_sequence_3fps.json"
        
    s3_seq_path = str(seq_path).replace("/Users/dxy/Desktop/warp", "s3://whoi-rsi-fish-detection/datasets")

    seq_manifest = {
        "seq-no": seq_num,
        "prefix": s3_dir,
        "number-of-frames": expected_frames,
        "frames": frame_list_dict
    }
     
    if not dry_run:
        with jsonlines.open(seq_path, mode='w') as writer:
            writer.write(seq_manifest)

    dataset_manifest.append(
        {"source-ref": s3_seq_path}
    )

    
dataset_manifest_path = "/Users/dxy/Desktop/warp/imerit_26102023_clips/28102023_manifest.json"
if not dry_run:
    with jsonlines.open(dataset_manifest_path, mode='w') as writer:
        for line in dataset_manifest:
            writer.write(line)
    print(f"created dataset manifest: {dataset_manifest_path}")    